What we need to do: 
1) Go through each newspaper and create a newspaper corpora, display the length of each corpora
2) Find either a) wikipedia dataset and train word2vec on it to compare non-context related words like "bad", 
"victim", "good", "attacker" to representation of each side of the conflict in the newspaper corpora, b) find a 
pretrained word2vec to do the same
3) Find a way to classify each article as pro-israel or pro-palestine

In [11]:
import json

# Open and read the JSON file
with open('data/data.json', 'r') as file:
    data = json.load(file)

# Print the data
# print(data)

Iterate over each newspaper dictionary {key: value} key = newspaper, value = articles associated with the newspaper

Use spacy for higher accuracy and nltk for efficiency

In [32]:
# we will need this to divide our articles into sentences
# import spacy
# nlp = spacy.load('en_core_web_trf') # Load the English language model
# print("model loaded")
# 
# if "parser" not in nlp.pipe_names:
#     sentencizer = nlp.create_pipe("sentencizer")
#     nlp.add_pipe(sentencizer, before="ner")
# 
# def divide_into_sentences(text):
#     doc = nlp(text)
#     return [sent.text for sent in doc.sents]
# 
# text = "Hello world! This is an example. NLP is interesting."
# sentences = divide_into_sentences(text)
# print(sentences)'

# import nltk
# nltk.download('punkt')  # Download required resources

from nltk.tokenize import sent_tokenize

def divide_into_sentences(text):
    return sent_tokenize(text)

# Example usage
text = "Hello world! This is an example. NLP is interesting."
sentences = divide_into_sentences(text)
print(sentences)


['Hello world!', 'This is an example.', 'NLP is interesting.']


In [37]:
# initialize an empty dictionary to have newspaper names as keys as, words published in that newspaper as value
newspaper_articles = {}

for newspaper, articles in data.items():
    print(f"Newspaper: {newspaper}")
    sentences = [] # List to hold texts of articles for this newspaper
    for article in articles: 
        text = article.get("text")
        if text:
            sentences.extend(divide_into_sentences(text))
   
    # tokenize sentences word by word         
    # Tokenize sentences word by word
    words = [sentence.split() for sentence in sentences]

    #add newspaper_name: texts from that newspaper as a new key value pair 
    newspaper_articles[newspaper] = words 
    # print the length of the corpus
    print("corpus size: " + str(len(newspaper_articles[newspaper])))

Newspaper: https://nytimes.com
corpus size: 4406
Newspaper: https://washingtonpost.com
corpus size: 3605
Newspaper: https://usatoday.com
corpus size: 10794
Newspaper: https://wsj.com
corpus size: 0
Newspaper: https://latimes.com
corpus size: 4160
Newspaper: https://nypost.com
corpus size: 3612
Newspaper: https://chicagotribune.com
corpus size: 3545
Newspaper: https://bostonglobe.com
corpus size: 10604
Newspaper: https://sfchronicle.com
corpus size: 0
Newspaper: https://dallasnews.com
corpus size: 7308
Newspaper: https://houstonchronicle.com
corpus size: 0
Newspaper: https://inquirer.com
corpus size: 0
Newspaper: https://startribune.com
corpus size: 18
Newspaper: https://seattletimes.com
corpus size: 0
Newspaper: https://miamiherald.com
corpus size: 0
Newspaper: https://freep.com
corpus size: 4324
Newspaper: https://denverpost.com
corpus size: 1645
Newspaper: https://orlandosentinel.com
corpus size: 3659
Newspaper: https://azcentral.com
corpus size: 3264
Newspaper: https://baltimoresun.

Print a newspaper corpus to control the output

In [38]:
print(newspaper_articles.get("https://nytimes.com"))

[['Card', '1', 'of', '305', 'Minutes', 'That', 'Will', 'Make', 'You', 'Love', 'Alice', 'ColtraneWe', 'asked', 'writers,', 'critics', 'and', 'musicians', 'including', 'Meshell', 'Ndegeocello', 'and', 'Angel', 'Bat', 'Dawid', 'to', 'tell', 'us', 'what', 'moves', 'them', 'in', 'Coltranes', 'spiritual', 'jazz.Background', 'Image:', 'A', 'colorful', 'animated', 'illustration', 'of', 'a', 'musician', 'with', 'a', 'loose', 'resemblance', 'to', 'Alice', 'Coltrane', 'playing', 'the', 'harp', 'with', 'their', 'eyes', 'closed.'], ['Submissions', 'are', 'open.Submissions', 'will', 'temporarily', 'close', 'starting', 'November', '4,', '2024', 'and', 'will', 'reopen', 'January', '3,', '2025.'], ['We', 'will', 'review', 'already', 'submitted', 'puzzles', 'during', 'that', 'period,', 'so', 'you', 'may', 'still', 'hear', 'from', 'us', 'while', 'submissions', 'are', 'closed.A', 'New', 'York', 'Times', 'crossword', 'will', 'be', 'a', 'collaboration', 'between', 'you', 'and', 'our', 'staff', 'of', 'editor

Train you model on the sentences. Revert back to hwo you preprocess data to increase accuracy. To dos for later:
1) better sentence division
2) more data
3) eliminate over-used words like "the", "a" etc.

Now,let's train our word2vec on NYT data, later we can create a loop to do so for every newspaper

In [46]:
from gensim.models import Word2Vec

# Prepare sentences for Word2Vec
sentences = newspaper_articles.get("https://nytimes.com")  # Each newspaper's corpus is one "document"
print(sentences)
# Train Word2Vec model
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

[['Card', '1', 'of', '305', 'Minutes', 'That', 'Will', 'Make', 'You', 'Love', 'Alice', 'ColtraneWe', 'asked', 'writers,', 'critics', 'and', 'musicians', 'including', 'Meshell', 'Ndegeocello', 'and', 'Angel', 'Bat', 'Dawid', 'to', 'tell', 'us', 'what', 'moves', 'them', 'in', 'Coltranes', 'spiritual', 'jazz.Background', 'Image:', 'A', 'colorful', 'animated', 'illustration', 'of', 'a', 'musician', 'with', 'a', 'loose', 'resemblance', 'to', 'Alice', 'Coltrane', 'playing', 'the', 'harp', 'with', 'their', 'eyes', 'closed.'], ['Submissions', 'are', 'open.Submissions', 'will', 'temporarily', 'close', 'starting', 'November', '4,', '2024', 'and', 'will', 'reopen', 'January', '3,', '2025.'], ['We', 'will', 'review', 'already', 'submitted', 'puzzles', 'during', 'that', 'period,', 'so', 'you', 'may', 'still', 'hear', 'from', 'us', 'while', 'submissions', 'are', 'closed.A', 'New', 'York', 'Times', 'crossword', 'will', 'be', 'a', 'collaboration', 'between', 'you', 'and', 'our', 'staff', 'of', 'editor

In [52]:
# Get the vector for a word
vector = model.wv["Israel"]

# Find most similar words
similar_words = model.wv.most_similar("Israel")
print(similar_words)

[('which', 0.9997478127479553), ('on', 0.9997339844703674), ('last', 0.9997299909591675), ('their', 0.9997296333312988), ('against', 0.9997272491455078), ('from', 0.9997257590293884), ('over', 0.9997256398200989), ('him', 0.9997245073318481), ('who', 0.9997239708900452), ('a', 0.9997195601463318)]


Currently, the similar vector are from high-frequency words like "and","which" etc. I had seen lists online that had 
all of these words that we should eliminate. I will find the list, get back on the problem and test out what will be 
the similar words. If the output is still unrelated, then I will see how many times the word we want to find out is 
mentioned in our corpus. If it is too low, we might have to expend the corpus. 
I might also use wikipedia corpus (news_unrelated) to test our hyptohesis. 